In [ ]:
import os
from dotenv import load_dotenv
from crewai import Agent, LLM, Task, Crew, Process

# Load environment variables
load_dotenv()

# Option D: Azure OpenAI
openai_api_key = os.getenv("AZURE_OPENAI_API_KEY")
openai_endpoint = os.getenv("AZURE_OPENAI_ENDPOINT")
openai_api_version = os.getenv("AZURE_OPENAI_API_VERSION")
openai_model_name = os.getenv("AZURE_OPENAI_MODEL_NAME")

llm = LLM(
    model="azure/gpt-4o-mini",
    api_key=openai_api_key,
    base_url=openai_endpoint,
    api_version=openai_api_version,
    azure=True
)

print("🚀 Environment configured!")
print(f"LLM Model: {llm.model}")

## Callbacks

In [7]:
from crewai import Agent

# AI Researcher Agent
research_agent = Agent(
    role="AI News Researcher",
    goal="Find and summarize the latest AI news from trusted sources",
    backstory="You are a dedicated AI journalist who follows the latest advancements in artificial intelligence.",
    llm=llm,
    verbose=False
)


In [8]:
import os

def notify_team(output):

    print(f"""Task Completed!
              Task: {output.description}
              Output Summary: {output.summary}""")
    
    # Create output directory if it doesn't exist
    os.makedirs("output", exist_ok=True)

    with open("output/latest_ai_news.txt", "w") as f:
        f.write(f"Task: {output.description}\n")
        f.write(f"Output Summary: {output.summary}\n")
        f.write(f"Full Output: {output.raw}\n")
    
    print("News summary saved to latest_ai_news.txt")

In [9]:
from crewai import Task

research_news_task = Task(
    description="Find and summarize the latest AI breakthroughs from the last week.",
    expected_output="A structured summary of AI news headlines.",
    agent=research_agent,
    callback=notify_team  # Calls the function after task completion
)


In [ ]:
from crewai import Crew

ai_news_crew = Crew(
    agents=[research_agent],
    tasks=[research_news_task],
    verbose=False
)

# Execute the workflow
result = ai_news_crew.kickoff()
